In [1]:
try:
    import google.colab  # noqa: F401

    %pip install -q dataeval maite-datasets
except Exception:
    pass

In [2]:
import numpy as np
import torch
import torch.nn as nn
from maite_datasets.object_detection import VOCDetection
from torchvision import models
from torchvision.transforms.v2 import GaussianNoise

from dataeval.data import Embeddings, Metadata
from dataeval.detectors.drift import DriftCVM, DriftKS, DriftMMD
from dataeval.metrics.bias import label_parity

# Set a random seed
rng = np.random.default_rng(213)

# Set default torch device for notebook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

In [3]:
# Define the embedding network
class EmbeddingNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Load in pretrained resnet18 model
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        # Add an additional fully connected layer with an embedding dimension of 128
        self.model.fc = nn.Linear(self.model.fc.in_features, 128)

    def forward(self, x):
        """Run input data through the model"""

        return self.model(x)

In [4]:
embedding_net = EmbeddingNet()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

 19%|█▉        | 8.50M/44.7M [00:00<00:00, 88.6MB/s]

 41%|████      | 18.1M/44.7M [00:00<00:00, 95.6MB/s]

 93%|█████████▎| 41.4M/44.7M [00:00<00:00, 163MB/s] 

100%|██████████| 44.7M/44.7M [00:00<00:00, 149MB/s]

In [5]:
# Load the training dataset
train_ds = VOCDetection("./data", year="2012", image_set="train", download=True)
print(train_ds)
print(f"Image 0 shape: {train_ds[0][0].shape}")

VOCDetection Dataset
--------------------
    Year: 2012
    Transforms: []
    Image_set: train
    Metadata: {'id': 'VOCDetection_train', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'train'}
    Path: /builds/jatic/aria/dataeval/docs/source/notebooks/data/vocdataset/VOCdevkit/VOC2012
    Size: 5717
Image 0 shape: (3, 442, 500)


In [6]:
# Load the "operational" dataset
operational_ds = VOCDetection("./data", year="2012", image_set="val", download=True)
print(operational_ds)
print(f"Image 0 shape: {train_ds[0][0].shape}")

VOCDetection Dataset
--------------------
    Year: 2012
    Transforms: []
    Image_set: val
    Metadata: {'id': 'VOCDetection_val', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'val'}
    Path: /builds/jatic/aria/dataeval/docs/source/notebooks/data/vocdataset/VOCdevkit/VOC2012
    Size: 5823
Image 0 shape: (3, 442, 500)


In [7]:
# Define pretrained model transformations
transforms = models.ResNet18_Weights.DEFAULT.transforms()

# Create training batches and targets
train_embs = Embeddings(train_ds, batch_size=64, model=embedding_net, transforms=transforms, cache=True)

# Create operational batches and targets
operational_embs = Embeddings(operational_ds, batch_size=64, model=embedding_net, transforms=transforms, cache=True)

In [8]:
print(f"({len(train_embs)}, {train_embs[0].shape})")  # (5717, shape)
print(f"({len(operational_embs)}, {operational_embs[0].shape})")  # (5823, shape)

(5717, torch.Size([128]))
(5823, torch.Size([128]))


In [9]:
# A type alias for all of the drift detectors
DriftDetector = DriftMMD | DriftCVM | DriftKS

# Create a mapping for the detectors to iterate over
detectors: dict[str, DriftDetector] = {
    "MMD": DriftMMD(train_embs),
    "CVM": DriftCVM(train_embs),
    "KS": DriftKS(train_embs),
}

In [10]:
train_embs.to_tensor()

tensor([[ 0.8497,  0.9974, -0.4607,  ...,  0.0153,  0.2364,  1.8828],
        [ 0.4045,  0.1001, -1.1110,  ..., -0.5731,  0.5290,  0.0664],
        [ 0.5055, -0.1768, -0.8227,  ..., -0.0903,  0.2561,  1.8796],
        ...,
        [ 0.5347,  0.2263, -0.9003,  ...,  1.5782,  0.4017,  0.8970],
        [ 0.5190,  0.5834, -0.8978,  ..., -0.1098,  0.2891,  0.4100],
        [-0.5420,  0.0498, -1.3946,  ..., -0.0788, -0.4771,  0.6135]],
       device='cuda:0')

In [11]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(operational_embs).drifted}")

MMD detected drift? False
CVM detected drift? False


KS detected drift? True


In [12]:
# Define transform with added gaussian noise
noisy_transforms = [transforms, GaussianNoise()]

# Applies gaussian noise to images before processing
noisy_embs = Embeddings(operational_ds, batch_size=64, model=embedding_net, transforms=noisy_transforms, cache=True)

In [13]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(noisy_embs).drifted}")

MMD detected drift? True


CVM detected drift? True


KS detected drift? True


In [14]:
# Get the metadata for each dataset
train_md = Metadata(train_ds)
operational_md = Metadata(operational_ds)

# The VOC dataset has 20 classes
label_parity(train_md.class_labels, operational_md.class_labels, num_classes=20).p_value

Processing datum metadata:   0%|          | 0/5717 [00:00<?, ?it/s]

Processing datum metadata:   0%|          | 0/5823 [00:00<?, ?it/s]

np.float64(0.949856067521638)